In [2]:
import pandas as pd

In [3]:
%matplotlib inline

## Read data and separate Switzerland, Cantons, Districts, Communes

### Load the Excel with all the data in it

In [15]:
df = pd.read_excel('ReadinData/px-x-0102010000_103 - Parteien.xlsx', sheetname='relrel')

In [16]:
df.head(2)

,Nummer,Einheit,BDP,CVP,EDU,EVP,FDP,GLP,Grüne,SP,SVP,Übrige
0,8100,Schweiz,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1,4551,...... Aadorf,-17.504673,72.846195,111.243296,8.748266,-38.426064,25.40505,-26.529719,-44.767115,36.936559,-96.692709


In [17]:
#Make sure we don't have any NaN in there
df = df[df['Einheit'].notnull()]

### Load some helper files

In [18]:
#Load the numbers and abbreviations file for the cantons
df_kant_ids = pd.read_csv('HelperData/Kantone-ABK-ID.csv')
df_kant_ids.head(2)

,Kanton_ABK,Kanton_ID,Kanton_Name,Kanton_Name2
0,ZH,1,Zürich,Zürich
1,BE,2,Bern,Bern / Berne


In [19]:
#Load the canton/district infos for the communes
df_gem_ids = pd.read_excel('HelperData/Gemeindestand-2015.xls')
df_gem_ids.head(2)

,Hist.-Nummer,Kanton,Bezirksnummer,Bezirksname,BFS Gde-nummer,Gemeindename,Datum der Aufnahme
0,13256,ZH,101,Affoltern,1,Aeugst am Albis,1976-11-15
1,11742,ZH,101,Affoltern,2,Affoltern am Albis,1960-01-01


In [20]:
df_gem_ids.pop('Hist.-Nummer')
df_gem_ids.pop('Gemeindename')
df_gem_ids.pop('Datum der Aufnahme')
df_gem_ids.rename(columns={'Bezirksnummer': 'Bezirk_ID', 'Bezirksname': 'Bezirk_Name', 'BFS Gde-nummer': 'Gemeinde_ID', 'Kanton': 'Kanton_ABK'}, inplace=True)
df_gem_ids.head(2)

,Kanton_ABK,Bezirk_ID,Bezirk_Name,Gemeinde_ID
0,ZH,101,Affoltern,1
1,ZH,101,Affoltern,2


In [21]:
#Construct a set of district ids
df_bez_ids = df_gem_ids.drop_duplicates(subset='Bezirk_ID')
df_bez_ids.pop('Gemeinde_ID')
df_bez_ids.head(2)

,Kanton_ABK,Bezirk_ID,Bezirk_Name
0,ZH,101,Affoltern
14,ZH,102,Andelfingen


## Get a dataframe for each geopraphic level

### Separate Switzerland

In [22]:
#Switzerland
df_ch = df[df['Einheit'] == 'Schweiz']
df_ch.pop('Einheit')
df_ch.pop('Nummer')
df_ch.head(2)

,BDP,CVP,EDU,EVP,FDP,GLP,Grüne,SP,SVP,Übrige
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df_ch.shape

(1, 10)

### Separate the cantons

In [13]:
#Cantons - there should be 26 of them
df_kant = df[df['Einheit'].str.contains('^- ')]
df_kant['Kanton_Name'] = df_kant['Einheit'].str.extract('^- (.*)', expand=True)
df_kant.head(2)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [15]:
#Add the ID and Abbreviations
df_kant = df_kant.merge(df_kant_ids, how='inner', left_on='Kanton_Name', right_on='Kanton_Name2')
df_kant.pop('Kanton_Name2')
df_kant.pop('Einheit')
df_kant.pop('Kanton_Name_y')
df_kant.pop('Nummer')
df_kant.rename(columns={'Kanton_Name_x': 'Kanton_Name'}, inplace=True)
df_kant.head(2)

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,Kanton_Name,Kanton_ABK,Kanton_ID
0,10.140814,10.189053,6.717105,4.113012,3.863790,1.440426,1.528121,0.158892,-0.562474,-2.143233,...,-7.446098,-7.251524,-12.883064,-12.916293,-9.923975,-11.694637,-17.112433,Zürich,ZH,1
1,-4.726403,-6.898074,-5.190242,-3.755251,-5.147559,-4.903439,-5.454057,-4.361319,-5.196198,-6.478511,...,19.993818,17.798107,22.045135,12.675966,22.641016,29.919495,18.411909,Bern / Berne,BE,2


In [16]:
df_kant.shape

(26, 104)

### Separate the districts

In [24]:
#Districts - there should be 148 of them
df_bez = df[df['Einheit'].str.contains('^>> ')]
df_bez['Bezirk_Name'] = df_bez['Einheit'].str.extract('^>> (.*)', expand=True)
df_bez['Bezirk_ID'] = df_bez['Nummer'].astype(int)
df_bez.pop('Nummer')
df_bez.pop('Einheit')
df_bez.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,BDP,CVP,EDU,EVP,FDP,GLP,Grüne,SP,SVP,Übrige,Bezirk_Name,Bezirk_ID
2325,35.885794,-75.890329,189.822990,113.305617,-46.314238,-50.193447,-4.555478,37.353166,24.497188,-22.688626,Arrondissement administratif Jura bernois,241
2326,8.333334,-53.290309,0.255768,143.318110,6.338554,31.987789,-7.884385,8.533223,8.017378,-58.376541,Bezirk Aarau,1901


In [25]:
df_bez.shape

(148, 12)

### Separate the communes

In [26]:
#Gemeinden - there should be 2324 of them
df_gem = df[df['Einheit'].str.contains('^\.\.\.\.\.\.')]
df_gem['Gemeinde_ID'] = df_gem['Nummer']
df_gem['Gemeinde_Name'] = df_gem['Einheit'].str.extract('^\.\.\.\.\.\. (.*)', expand=True)
df_gem.pop('Nummer')
df_gem.pop('Einheit')
df_gem.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,BDP,CVP,EDU,EVP,FDP,GLP,Grüne,SP,SVP,Übrige,Gemeinde_ID,Gemeinde_Name
1,-17.504673,72.846195,111.243296,8.748266,-38.426064,25.405050,-26.529719,-44.767115,36.936559,-96.692709,4551,Aadorf
2,-11.602571,-54.139354,-38.232626,99.063276,10.369443,70.622188,25.372588,40.593077,-22.995957,-47.999942,4001,Aarau


In [27]:
df_gem.shape

(2324, 12)

## Export the data

In [28]:
#Switzerland
df_ch.to_csv('StructuredData/CH-Parteien.csv', index=False)
df_temp = pd.read_csv('StructuredData/CH-Parteien.csv')
df_temp.head(2)

,BDP,CVP,EDU,EVP,FDP,GLP,Grüne,SP,SVP,Übrige
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#Cantons
df_kant.to_csv('StructuredData/kant-Parteien.csv', index=False)
df_temp = pd.read_csv('StructuredData/kant-Parteien.csv')
df_temp.head(2)

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,Kanton_Name,Kanton_ABK,Kanton_ID
0,10.140814,10.189053,6.717105,4.113012,3.863790,1.440426,1.528121,0.158892,-0.562474,-2.143233,...,-7.446098,-7.251524,-12.883064,-12.916293,-9.923975,-11.694637,-17.112433,Zürich,ZH,1
1,-4.726403,-6.898074,-5.190242,-3.755251,-5.147559,-4.903439,-5.454057,-4.361319,-5.196198,-6.478511,...,19.993818,17.798107,22.045135,12.675966,22.641016,29.919495,18.411909,Bern / Berne,BE,2


In [29]:
#Districts
df_bez.to_csv('StructuredData/bez-Parteien.csv', index=False)
df_temp = pd.read_csv('StructuredData/bez-Parteien.csv')
df_temp.head(2)

,BDP,CVP,EDU,EVP,FDP,GLP,Grüne,SP,SVP,Übrige,Bezirk_Name,Bezirk_ID
0,35.885794,-75.890329,189.822990,113.305617,-46.314238,-50.193447,-4.555478,37.353166,24.497188,-22.688626,Arrondissement administratif Jura bernois,241
1,8.333334,-53.290309,0.255768,143.318110,6.338554,31.987789,-7.884385,8.533223,8.017378,-58.376541,Bezirk Aarau,1901


In [30]:
#Communes
df_gem.to_csv('StructuredData/gem-Parteien.csv', index=False)
df_temp = pd.read_csv('StructuredData/gem-Parteien.csv')
df_temp.head(2)

,BDP,CVP,EDU,EVP,FDP,GLP,Grüne,SP,SVP,Übrige,Gemeinde_ID,Gemeinde_Name
0,-17.504673,72.846195,111.243296,8.748266,-38.426064,25.405050,-26.529719,-44.767115,36.936559,-96.692709,4551,Aadorf
1,-11.602571,-54.139354,-38.232626,99.063276,10.369443,70.622188,25.372588,40.593077,-22.995957,-47.999942,4001,Aarau
